In [ ]:
import numpy as np
import cv2
import os

import matplotlib.pyplot as plt
%matplotlib inline

###  Define handy functions

**Useful vis function**

In [ ]:
def show_image(orig_img, count):
    fig, ax = plt.subplots(figsize=(20, 20))
    _ = ax.set_title('Number of objects is {}'.format(count))
    _ = ax.axis('off')
    _ = ax.imshow(orig_img)
    plt.show()

In [ ]:
def count_objects(img):
    
    # equalization
    clahe = cv2.createCLAHE(clipLimit=1.25, tileGridSize=(7, 7))
    img = clahe.apply(img)
    
    # denoising 
    img = cv2.medianBlur(img, 3)
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 13, 13)
    img = cv2.GaussianBlur(img, (3, 3), 1)
    _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # separate objects
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, iterations=1)

    # post processing
    img = cv2.GaussianBlur(img, (3, 3), 1)
    img = cv2.fastNlMeansDenoising(img, None, 70, 15, 15)
    _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # get separated objects
    _, markers = cv2.connectedComponents(img) 
    
    # count classes
    count = np.unique(markers.flatten()).shape[0] - 1
    
    return count
  

In [ ]:
for root, dir, files in os.walk('./data/count/'):
    img_paths = [os.path.join(root, file) for file in files]

In [ ]:
for img_path in img_paths:
    
    # read image
    img = cv2.imread(img_path)
    
    # get original image in RGB for visualization
    orig_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # get image in grayscale for further processing
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # get count
    count = count_objects(img)
    
    # show results
    show_image(orig_img, count)